# GPT-4 Vision Relabeled Test

Test accuracy of GPT-4 Vision on relabeled test set.

In [83]:
# Packages
import base64
import requests
import openai
import os
import pandas as pd

In [84]:
# Get and set API key
with open(os.path.expanduser('~') + '/Documents/OpenAI/computer-vision-project.txt', 'r') as file:
    api_key = file.read()
openai.api_key = api_key

In [85]:
# test run or not
test_run = True

## Load Data

In [86]:
# Load "relabeled_test_no_blur_old_and_new_labels.xlsx"
relabeled_test_no_blur_old_and_new_labels = pd.read_excel('../../../Data/Relabeled_Test_No_Blur/relabeled_test_no_blur_old_and_new_labels.xlsx')

# Delete if 'New Class' is not 'SUV', 'Sedan', 'Pickup', or 'Convertible'
relabeled_test_no_blur_old_and_new_labels = relabeled_test_no_blur_old_and_new_labels[relabeled_test_no_blur_old_and_new_labels['New Class'].isin(['SUV', 'Sedan', 'Pickup', 'Convertible'])]

# For testing, limit to 5 rows
if test_run:
    relabeled_test_no_blur_old_and_new_labels = relabeled_test_no_blur_old_and_new_labels.head(5)

# Delete if 'New Class' is not 'SUV', 'Sedan', 'Pickup', or 'Convertible'
relabeled_test_no_blur_old_and_new_labels = relabeled_test_no_blur_old_and_new_labels[relabeled_test_no_blur_old_and_new_labels['New Class'].isin(['SUV', 'Sedan', 'Pickup', 'Convertible'])]

# Construct image path
relabeled_test_no_blur_old_and_new_labels['non_blurred_image_path'] = '../../../Images/test/No Blur/' + relabeled_test_no_blur_old_and_new_labels['filename']

# Print the first few rows of the dataframe
print(relabeled_test_no_blur_old_and_new_labels.head())

                                            filename  label    New Class  \
0  Convertible_test_orig_test_00023_resized_no_bl...      2  Convertible   
1  Convertible_test_orig_test_00096_resized_no_bl...      2  Convertible   
2  Convertible_test_orig_test_00107_resized_no_bl...      2  Convertible   
3  Convertible_test_orig_test_00135_resized_no_bl...      2  Convertible   
4  Convertible_test_orig_test_00147_resized_no_bl...      2  Convertible   

     Old Class                             non_blurred_image_path  
0  Convertible  ../../../Images/test/No Blur/Convertible_test_...  
1  Convertible  ../../../Images/test/No Blur/Convertible_test_...  
2  Convertible  ../../../Images/test/No Blur/Convertible_test_...  
3  Convertible  ../../../Images/test/No Blur/Convertible_test_...  
4  Convertible  ../../../Images/test/No Blur/Convertible_test_...  


## Image Processing

In [87]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Function to classify the image
def classify_image(image_path):

  # Getting the base64 string
  base64_image = encode_image(image_path)

  # Setting the headers
  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  # Create payload
  payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "Please classify the image as one of the following classes: SUV, Sedan, Pickup, or Convertible. Return only the word SUV, Sedan, Pickup, or Convertible."
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  return response.json()

In [88]:
# Test on one image
response = classify_image(relabeled_test_no_blur_old_and_new_labels['non_blurred_image_path'].iloc[0])
response

{'id': 'chatcmpl-95ozZpJWIHtuh3pXsK9irEFlKdxPH',
 'object': 'chat.completion',
 'created': 1711174681,
 'model': 'gpt-4-1106-vision-preview',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'Convertible'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 299, 'completion_tokens': 1, 'total_tokens': 300},
 'system_fingerprint': None}

In [89]:
response['choices'][0]['message']['content']

'Convertible'

## Classify Images

In [90]:
# Iterate over rows of relabeled_test_no_blur_old_and_new_labels
for index, row in relabeled_test_no_blur_old_and_new_labels.iterrows():
    # Classify the image
    response = classify_image(row['non_blurred_image_path'])
    # Get the classification
    classification = response['choices'][0]['message']['content']
    # Add the classification to the dataframe
    relabeled_test_no_blur_old_and_new_labels.loc[index, 'gpt_4_v_classification'] = classification

relabeled_test_no_blur_old_and_new_labels

,filename,label,New Class,Old Class,non_blurred_image_path,gpt_4_v_classification
0,Convertible_test_orig_test_00023_resized_no_bl...,2,Convertible,Convertible,../../../Images/test/No Blur/Convertible_test_...,Convertible
1,Convertible_test_orig_test_00096_resized_no_bl...,2,Convertible,Convertible,../../../Images/test/No Blur/Convertible_test_...,Convertible
2,Convertible_test_orig_test_00107_resized_no_bl...,2,Convertible,Convertible,../../../Images/test/No Blur/Convertible_test_...,Sedan
3,Convertible_test_orig_test_00135_resized_no_bl...,2,Convertible,Convertible,../../../Images/test/No Blur/Convertible_test_...,Convertible
4,Convertible_test_orig_test_00147_resized_no_bl...,2,Convertible,Convertible,../../../Images/test/No Blur/Convertible_test_...,Convertible


## Export Results

In [91]:
relabeled_test_no_blur_old_and_new_labels.to_excel('../../../Data/Predictions/GPT-4 Vision/GPT-4_Vision_Test_No_Blur_predictions.xlsx', index=False)
